In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.metrics import classification_report, confusion_matrix

import tensorflow as tf
import keras
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import applications
from keras.models import Sequential, load_model
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Flatten, Dense, Dropout
from keras.preprocessing import image

import warnings
warnings.filterwarnings('ignore')

In [2]:
train_image='D:/Documents/Capstone/Project -2/Dataset/sheath_blight_Vs_non-sheath_blight/train'

In [3]:
train_datagen=ImageDataGenerator(rescale=1./255,
                                 zoom_range=0.2,
                                 shear_range=0.1,
                                vertical_flip=False)

In [4]:
training_set=train_datagen.flow_from_directory(train_image,
                                              target_size=(224,224),
                                              class_mode='categorical',
                                              batch_size=5)

Found 1358 images belonging to 2 classes.


In [5]:
test_datagen = ImageDataGenerator(rescale = 1./255,
                                 zoom_range=0.2)

test_set=train_datagen.flow_from_directory('D:/Documents/Capstone/Project -2/Dataset/sheath_blight_Vs_non-sheath_blight/test',
                                              target_size=(224,224),
                                              class_mode='categorical',
                                              batch_size=5)

Found 72 images belonging to 2 classes.


In [6]:
base_model = tf.keras.applications.InceptionResNetV2(
                     include_top=False,
                     weights='imagenet',
                     input_shape=(224,224,3)
                     )
 
base_model.trainable=False
 
model = tf.keras.Sequential([
        base_model,  
        tf.keras.layers.BatchNormalization(renorm=True),
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(2, activation='softmax')
    ])

In [7]:
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inception_resnet_v2 (Functi  (None, 5, 5, 1536)       54336736  
 onal)                                                           
                                                                 
 batch_normalization_203 (Ba  (None, 5, 5, 1536)       10752     
 tchNormalization)                                               
                                                                 
 global_average_pooling2d (G  (None, 1536)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 512)               786944    
                                                                 
 dense_1 (Dense)             (None, 256)               131328    
                                                        

In [9]:
early = tf.keras.callbacks.EarlyStopping(patience=10, min_delta=0.001, restore_best_weights=True)

In [10]:
batch_size=32
STEP_SIZE_TRAIN = training_set.n//training_set.batch_size
STEP_SIZE_VALID = test_set.n//test_set.batch_size

# fit model
history = model.fit(training_set, steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=test_set,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=7,
                    callbacks=[early])

Epoch 1/7
271/271 [==============================] - 563s 2s/step - loss: 0.3829 - accuracy: 0.8426 - val_loss: 0.2373 - val_accuracy: 0.9286
Epoch 2/7
271/271 [==============================] - 495s 2s/step - loss: 0.1660 - accuracy: 0.9468 - val_loss: 0.1556 - val_accuracy: 0.9429
Epoch 3/7
271/271 [==============================] - 524s 2s/step - loss: 0.0909 - accuracy: 0.9712 - val_loss: 0.0800 - val_accuracy: 0.9714
Epoch 4/7
271/271 [==============================] - 563s 2s/step - loss: 0.0714 - accuracy: 0.9682 - val_loss: 0.0782 - val_accuracy: 0.9857
Epoch 5/7
271/271 [==============================] - 619s 2s/step - loss: 0.0806 - accuracy: 0.9727 - val_loss: 0.0166 - val_accuracy: 1.0000
Epoch 6/7
271/271 [==============================] - 710s 3s/step - loss: 0.0622 - accuracy: 0.9852 - val_loss: 0.1836 - val_accuracy: 0.9429
Epoch 7/7
271/271 [==============================] - 810s 3s/step - loss: 0.0744 - accuracy: 0.9786 - val_loss: 0.1276 - val_accuracy: 0.9857
